This notebook contains the code for training models whether it is a new model or to continue one

#Setup

##Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import sys
sys.path.append('/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch')

import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import json
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
from scipy import spatial
import json
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
import torchvision.transforms.functional as F
from matplotlib import cm as CM
import random
from torchvision import datasets, transforms
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],          std=[0.229, 0.224, 0.225]),
                   ])
                          
%matplotlib inline

Mounted at /content/gdrive


##Paths

In [ ]:
root = '/content/gdrive/My Drive/TFE_crowd_counting'
dataset_path = os.path.join(root,'Dataset')
images_path = os.path.join(dataset_path,'Basic datas/images/A10_small')
path_sets = [images_path]  #All images at the same place
scripts_path = os.path.join(root,'CSRNet-pytorch')

img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.png')):
        img_paths.append(img_path)

print(img_paths)

['/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10_small/A10_small_1.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10_small/A10_small_6.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10_small/A10_small_5.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10_small/A10_small_7.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10_small/A10_small_8.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10_small/A10_small_9.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10_small/A10_small_10.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10_small/A10_small_3.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10_small/A10_small_2.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10_small/A10_small_11.png

# Mix A10 & Shanghai

In [ ]:
root = '/content/gdrive/My Drive/TFE_crowd_counting'
A10_dataset_path = os.path.join(root,'Dataset')
A10_images_path = os.path.join(A10_dataset_path,'Basic datas/images/A10')
A10_path_sets = [A10_images_path]  #All images at the same place

sh_dataset_path = os.path.join(root,'part_A_final')
sh_images_path_train = os.path.join(sh_dataset_path,'train_data/images')
sh_path_sets_train = [sh_images_path_train]  #All images at the same place

sh_images_path_test = os.path.join(sh_dataset_path,'test_data/images')
sh_path_sets_test = [sh_images_path_test]  #All images at the same place

scripts_path = os.path.join(root,'CSRNet-pytorch')

A10_img_paths = []
for path in A10_path_sets:
    for img_path in glob.glob(os.path.join(path, '*.png')):
        A10_img_paths.append(img_path)

sh_img_paths_train = []
for path in sh_path_sets_train:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        sh_img_paths_train.append(img_path)

sh_img_paths_test = []
for path in sh_path_sets_test:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        sh_img_paths_test.append(img_path)



print(A10_img_paths)
print(sh_img_paths_train)
print(sh_img_paths_test)

['/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10/A10_17.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10/A10_18.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10/A10_19.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10/A10_20.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10/A10_21.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10/A10_22.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10/A10_23.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10/A10_24.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10/A10_25.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10/A10_26.png', '/content/gdrive/My Drive/TFE_crowd_counting/Dataset/Basic datas/images/A10/A10_5.png', '/content/gdrive/My D

In [ ]:
#Json writing for train (75% of images) and test (25% of images)
json_paths = []
for path in A10_path_sets:
    for json_path in glob.glob(os.path.join(path, '*.png')):
        json_paths.append(json_path)
random.shuffle(json_paths)
l =int(3/4 * len(json_paths))
pathstrain = json_paths[:l]
pathstests = json_paths[l:]

train_array = sh_img_paths_train + pathstrain
test_array = sh_img_paths_test + pathstests

random.shuffle(train_array)
random.shuffle(test_array)

with open(os.path.join(scripts_path,'Model_train.json'), 'w', encoding='utf-8') as f:
    json.dump(train_array, f, ensure_ascii=False, indent=4)

with open(os.path.join(scripts_path,'Model_test.json'), 'w', encoding='utf-8') as f:
    json.dump(test_array, f, ensure_ascii=False, indent=4)


#Json generation
In this part we generate the Json files with the paths to the train/test images used to train the model

In [ ]:
#Json writing for train (75% of images) and test (25% of images)
json_paths = []
for path in path_sets:
    for json_path in glob.glob(os.path.join(path, '*.png')):
        json_paths.append(json_path)
random.shuffle(json_paths)
l =int(3/4 * len(json_paths))
pathstrain = json_paths[:l]
pathstests = json_paths[l:]

with open(os.path.join(scripts_path,'Model_train.json'), 'w', encoding='utf-8') as f:
    json.dump(pathstrain, f, ensure_ascii=False, indent=4)

with open(os.path.join(scripts_path,'Model_test.json'), 'w', encoding='utf-8') as f:
    json.dump(pathstests, f, ensure_ascii=False, indent=4)


#Training

To continue a training add this argument between the "0 0" at the end : --pre /content/gdrive/'My Drive'/'TFE_crowd_counting'/CSRNet-pytorch/models/PartAmodel_best.pth.tar

In [ ]:
!python /content/gdrive/'My Drive'/'TFE_crowd_counting'/CSRNet-pytorch/train.py /content/gdrive/'My Drive'/'TFE_crowd_counting'/CSRNet-pytorch/Model_train.json /content/gdrive/'My Drive'/'TFE_crowd_counting'/CSRNet-pytorch/Model_test.json 0 0

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
epoch 0, processed 0 samples, lr 0.0000001000
Epoch: [0][0/1524]	Time 1.011 (1.011)	Data 0.607 (0.607)	Loss 54.9022 (54.9022)	
Epoch: [0][30/1524]	Time 0.969 (0.983)	Data 0.833 (0.853)	Loss 391.8687 (341.9657)	
Epoch: [0][60/1524]	Time 0.780 (0.905)	Data 0.685 (0.772)	Loss 28.8515 (267.2844)	
Epoch: [0][90/1524]	Time 0.965 (0.909)	Data 0.834 (0.772)	Loss 351.9498 (322.4476)	
Epoch: [0][120/1524]	Time 0.791 (0.892)	Data 0.656 (0.748)	Loss 124.8562 (278.8311)	
Epoch: [0][150/1524]	Time 0.757 (0.877)	Data 0.616 (0.727)	Loss 186.1075 (300.4742)	
Epoch: [0][180/1524]	Time 0.768 (0.853)	Data 0.650 (0.697)	Loss 86.6090 (348.4710)	
Epoch: [0][210/1524]	Time 0.773 (0.856)	Data 0.631 (0.689)	Loss 803.8731 (343.2418)	
Epoch: [0][240/1524]	Time 0.825 (0.840)	Data 0.764 (0.656)	Loss 20.1741 (322.5586)

#Training in python

In [ ]:
import warnings
from utils import save_checkpoint
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import argparse
import dataset
import cv2
import time

class arg:
  def __init__(self):
    self.x = 3
#Autre fonction que MSELOSS ou autre optimizer que SGD peuvent etre implémentés
def complete_train(datasetpath, modelpath = None, shuffle = True, gpu = True, init_lr = 1e-7, batch_size = 1, momentum = 0.95, decay = 5*1e-4,epochs = 400, workers = 4):
    global args,best_prec1
    
    best_prec1 = 1e6
    args = arg()
    args.original_lr = init_lr
    args.lr = 1e-7
    args.batch_size = batch_size
    args.gpu = gpu
    args.momentum      = momentum
    args.decay         = decay
    args.start_epoch   = 0
    args.epochs = epochs
    args.steps         = [-1,1,100,150]
    args.scales        = [1,1,1,1]
    args.workers = workers
    args.seed = time.time()
    args.print_freq = 30

    for json_path in glob.glob(os.path.join(datasetpath, '*.jpg')):
        json_paths.append(json_path)
    if shuffle:
      random.shuffle(json_paths)
    l =int(3/4 * len(json_paths))
    train_list = json_paths[:l]
    val_list = json_paths[l:]
    
    #os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    #torch.cuda.manual_seed(args.seed)
    
    model = CSRNet()
    
    model = model.cuda()
    
    criterion = nn.MSELoss(size_average=False).cuda() #loss function
    
    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                weight_decay=args.decay)

    if modelpath is not None:
        if os.path.isfile(modelpath):
            print("=> loading checkpoint '{}'".format(args.pre))
            checkpoint = torch.load(args.pre)
            args.start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.pre, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.pre))
 
    for epoch in range(args.start_epoch, args.epochs):
        
        adjust_learning_rate(optimizer, epoch)
        
        train(train_list, model, criterion, optimizer, epoch)
        prec1 = validate(val_list, model, criterion)
        
        is_best = prec1 < best_prec1
        best_prec1 = min(prec1, best_prec1)
        print(' * best MAE {mae:.3f} '
              .format(mae=best_prec1))
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': args.pre,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer' : optimizer.state_dict(),
        }, is_best,args.task)

def train(train_list, model, criterion, optimizer, epoch):
    
    losses = AverageMeter()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    
    
    train_loader = torch.utils.data.DataLoader(
        dataset.listDataset(train_list,
                       shuffle=True,
                       transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ]), 
                       train=True, 
                       seen=model.seen,
                       batch_size=args.batch_size,
                       num_workers=args.workers),
        batch_size=args.batch_size)
    print('epoch %d, processed %d samples, lr %.10f' % (epoch, epoch * len(train_loader.dataset), args.lr))
    
    model.train()
    end = time.time()
    
    for i,(img, target)in enumerate(train_loader):
        data_time.update(time.time() - end)
        
        img = img.cuda()
        img = Variable(img)
        output = model(img)
        
        
        
        
        target = target.type(torch.FloatTensor).unsqueeze(0).cuda()
        target = Variable(target)
        
        
        loss = criterion(output, target)
        
        losses.update(loss.item(), img.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        
        batch_time.update(time.time() - end)
        end = time.time()
        
        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  .format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses))
    
def validate(val_list, model, criterion):
    print ('begin test')
    test_loader = torch.utils.data.DataLoader(
    dataset.listDataset(val_list,
                   shuffle=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ]),  train=False),
    batch_size=args.batch_size)    
    
    model.eval()
    
    mae = 0
    
    for i,(img, target) in enumerate(test_loader):
        img = img.cuda()
        img = Variable(img)
        output = model(img)
        
        mae += abs(output.data.sum()-target.sum().type(torch.FloatTensor).cuda())
        
    mae = mae/len(test_loader)    
    print(' * MAE {mae:.3f} '
              .format(mae=mae))

    return mae    
        
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    
    
    args.lr = args.original_lr
    
    for i in range(len(args.steps)):
        
        scale = args.scales[i] if i < len(args.scales) else 1
        
        
        if epoch >= args.steps[i]:
            args.lr = args.lr * scale
            if epoch == args.steps[i]:
                break
        else:
            break
    for param_group in optimizer.param_groups:
        param_group['lr'] = args.lr
        
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count   

In [ ]:
complete_train(images_path)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch 0, processed 0 samples, lr 0.0000001000
Epoch: [0][0/528]	Time 0.687 (0.687)	Data 0.079 (0.079)	Loss 45.3025 (45.3025)	
Epoch: [0][30/528]	Time 1.332 (1.301)	Data 0.049 (0.048)	Loss 3.5069 (24.6324)	
Epoch: [0][60/528]	Time 1.355 (1.325)	Data 0.047 (0.048)	Loss 1.9449 (13.1200)	


KeyboardInterrupt: ignored